In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 4}
----------
[[ 1377  5340]
 [    0 17377]]
----------
              precision    recall  f1-score   support

           0       1.00      0.21      0.34      6717
           1       0.76      1.00      0.87     17377

    accuracy                           0.78     24094
   macro avg       0.88      0.60      0.60     24094
weighted avg       0.83      0.78      0.72     24094

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.23985898, 0.28174853, 0.3650229 , 0.42386568, 0.38147962,
        0.41887987, 0.50365329, 0.54204953, 0.5151217 , 0.56798041,
        0.64128494, 0.66123044, 0.64278007, 0.63679671, 0.77642477,
        0.23985815, 0.33161271, 0.34657335, 0.43234396, 0.51312768,
        0.51518309, 0.61186349, 0.71159708, 0.72555983, 0.78290713,
        0.81631637, 0.89560521, 0.86468816, 0.8487283 , 0.90710568]),
 'std_fit_time': array([0.00847781, 0.00149536, 0.01396346, 0.01795232, 0.00548589,
        0.01097047, 0.01097131, 0.00748074, 0.02144241, 0.03041899,
        0.03989291, 0.03690135, 0.01246786, 0.00249338, 0.05435455,
        0.00049806, 0.01745307, 0.00748014, 0.0154593 , 0.01346338,
        0.00255311, 0.01845086, 0.02144325, 0.05235946, 0.01097119,
        0.00348997, 0.02892292, 0.00099659, 0.00997317, 0.03437805]),
 'mean_score_time': array([0.01595783, 0.01496089, 0.01645708, 0.0144614 , 0.01346385,
        0.01695454, 0.0134635 , 0.01894915, 0.01994634, 0.01